In [86]:
import pandas as pd
import numpy as np

In [87]:
%%time
flights = pd.read_csv('flights_v5.csv')

CPU times: user 22.6 s, sys: 6.76 s, total: 29.3 s
Wall time: 35.7 s


In [88]:
%%time
passengers = pd.read_csv('cleaned_passengers_data.csv')

CPU times: user 644 ms, sys: 211 ms, total: 856 ms
Wall time: 939 ms


In [89]:
flights = flights.drop(columns = ['Unnamed: 0'])
passengers = passengers.drop(columns = ['Unnamed: 0'])

In [90]:
print('passengers shape: {}, flights shape: {}'.format(passengers.shape,flights.shape))

passengers shape: (633799, 10), flights shape: (15351709, 14)


In [91]:
pd.set_option('display.max_columns',40)

In [92]:
passengers.head(2)

,percent_pload_pass,percent_capacity,distance,accel_time,unique_carrier,origin_airport_id,route,dest_airport_id,aircraft_group,date
0,0.549322,0.777778,86.0,5.0,KAH,10559,BFI-ESD,11588,4,2018-1
1,0.354563,0.500000,69.0,5.0,KAH,10844,BYW-LKE,13002,0,2018-1


In [93]:
flights.head(2)

,fl_date,mkt_carrier,mkt_carrier_fl_num,origin_airport_id,dest_airport_id,route,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,crs_route,time_diff_crs_elapsed_route_time,speed
0,2018-01-01,DL,3468,15380,11433,TVC-DTW,06:15:00,07:38:00,83,207,-17,83,0,5.914286
1,2018-01-01,DL,3470,13871,11433,OMA-DTW,08:15:00,11:21:00,126,651,58,186,60,7.397727


In [94]:
flights['fl_date_split'] = flights['fl_date'].str.split('-')

In [95]:
flights.head(2)

,fl_date,mkt_carrier,mkt_carrier_fl_num,origin_airport_id,dest_airport_id,route,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,crs_route,time_diff_crs_elapsed_route_time,speed,fl_date_split
0,2018-01-01,DL,3468,15380,11433,TVC-DTW,06:15:00,07:38:00,83,207,-17,83,0,5.914286,"[2018, 01, 01]"
1,2018-01-01,DL,3470,13871,11433,OMA-DTW,08:15:00,11:21:00,126,651,58,186,60,7.397727,"[2018, 01, 01]"


In [96]:
def format_split(x):
    x = x[0] + '-' + x[1]
    return x

In [97]:
flights['fl_date_split']= flights['fl_date_split'].map(format_split)

In [98]:
flights = flights.rename(columns={'fl_date_split':'fl_year_mon'})

In [99]:
common_carrier =  passengers[passengers.unique_carrier.isin(np.unique(flights.mkt_carrier))]

In [100]:
common_or_id = common_carrier[common_carrier.origin_airport_id.isin(flights.origin_airport_id)]

In [101]:
common_des_id = common_or_id[common_or_id.dest_airport_id.isin(flights.dest_airport_id)]

In [102]:
common_route = common_des_id[common_des_id.route.isin(flights.route)]

## filtering and merging based on dates for passengers and flights

### columns to merge on :

In [103]:
left_cols = ['unique_carrier','origin_airport_id','route','dest_airport_id']
right_cols = ['mkt_carrier','origin_airport_id','route','dest_airport_id']

### 2018

##### 2018-01

In [104]:
common_test= common_route[common_route['date'].str.contains("2018-1",case=False)]
common_test= common_test[common_test['date'].str.len()==6]
flights_18_01 = flights[flights['fl_year_mon'].str.contains("2018-01")]
merge_18_01 = pd.merge(left=common_test, right = flights_18_01, how='inner',left_on = left_cols,right_on=right_cols)
merge_18_01 = merge_18_01.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

In [105]:
merge_18_01

,percent_pload_pass,percent_capacity,accel_time,route,aircraft_group,fl_date,mkt_carrier,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,distance_y,arr_delay,crs_route,time_diff_crs_elapsed_route_time,speed
0,0.676118,0.805369,27.0,DSM-MSP,6,2018-01-01,DL,3405,13:20:00,14:42:00,82,232,92,82,0,5.523810
1,0.676118,0.805369,27.0,DSM-MSP,6,2018-01-01,DL,3756,06:45:00,08:09:00,84,232,244,84,0,5.395349
2,0.676118,0.805369,27.0,DSM-MSP,6,2018-01-01,DL,3850,17:19:00,18:50:00,91,232,11,91,0,5.658537
3,0.676118,0.805369,27.0,DSM-MSP,6,2018-01-02,DL,3671,12:05:00,13:20:00,75,232,-20,75,0,5.523810
4,0.676118,0.805369,27.0,DSM-MSP,6,2018-01-02,DL,3756,06:45:00,08:09:00,84,232,4,84,0,5.272727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233554,0.417261,0.824280,12883.0,OGG-HNL,6,2018-01-31,HA,505,14:47:00,15:29:00,42,100,21,42,0,4.347826
1233555,0.417261,0.824280,12883.0,OGG-HNL,6,2018-01-31,HA,515,23:08:00,23:44:00,36,100,-5,36,0,4.545455
1233556,0.417261,0.824280,12883.0,OGG-HNL,6,2018-01-31,HA,545,19:02:00,19:40:00,38,100,-7,38,0,4.000000
1233557,0.417261,0.824280,12883.0,OGG-HNL,6,2018-01-31,HA,555,21:57:00,22:34:00,37,100,-12,37,0,4.347826


##### 2018-02

In [ ]:
common_test= common_route[common_route['date'].str.contains("2018-2",case=False)]
flights_18_02 = flights[flights['fl_year_mon'].str.contains("2018-02")]
merge_18_02 = pd.merge(left=common_test, right = flights_18_02, how='inner',left_on = left_cols,right_on=right_cols)
merge_18_02 = merge_18_02.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

In [ ]:
merge_18_02

##### 2018-03

In [ ]:
common_test= common_route[common_route['date'].str.contains("2018-3",case=False)]
flights_18_03 = flights[flights['fl_year_mon'].str.contains("2018-03")]
merge_18_03 = pd.merge(left=common_test, right = flights_18_03, how='inner',left_on = left_cols,right_on=right_cols)
merge_18_03 = merge_18_03.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2018-04

In [ ]:
common_test= common_route[common_route['date'].str.contains("2018-4",case=False)]
flights_18_04 = flights[flights['fl_year_mon'].str.contains("2018-04")]
merge_18_04 = pd.merge(left=common_test, right = flights_18_04, how='inner',left_on = left_cols,right_on=right_cols)
merge_18_04 = merge_18_04.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2018-05

In [ ]:
common_test= common_route[common_route['date'].str.contains("2018-5",case=False)]
flights_18_05 = flights[flights['fl_year_mon'].str.contains("2018-05")]
merge_18_05 = pd.merge(left=common_test, right = flights_18_05, how='inner',left_on = left_cols,right_on=right_cols)
merge_18_05 = merge_18_05.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2018-06

In [ ]:
common_test= common_route[common_route['date'].str.contains("2018-6",case=False)]
flights_18_06 = flights[flights['fl_year_mon'].str.contains("2018-06")]
merge_18_06 = pd.merge(left=common_test, right = flights_18_06, how='inner',left_on = left_cols,right_on=right_cols)
merge_18_06 = merge_18_06.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2018-07

In [ ]:
common_test= common_route[common_route['date'].str.contains("2018-7",case=False)]
flights_18_07 = flights[flights['fl_year_mon'].str.contains("2018-07")]
merge_18_07 = pd.merge(left=common_test, right = flights_18_07, how='inner',left_on = left_cols,right_on=right_cols)
merge_18_07 = merge_18_07.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2018-08

In [ ]:
common_test= common_route[common_route['date'].str.contains("2018-8",case=False)]
flights_18_08 = flights[flights['fl_year_mon'].str.contains("2018-08")]
merge_18_08 = pd.merge(left=common_test, right = flights_18_08, how='inner',left_on = left_cols,right_on=right_cols)
merge_18_08 = merge_18_08.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2018-09

In [ ]:
common_test= common_route[common_route['date'].str.contains("2018-9",case=False)]
flights_18_09 = flights[flights['fl_year_mon'].str.contains("2018-09")]
merge_18_09 = pd.merge(left=common_test, right = flights_18_09, how='inner',left_on = left_cols,right_on=right_cols)
merge_18_09 = merge_18_09.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2018-10

In [ ]:
common_test= common_route[common_route['date'].str.contains("2018-10",case=False)]
flights_18_10 = flights[flights['fl_year_mon'].str.contains("2018-10")]
merge_18_10 = pd.merge(left=common_test, right = flights_18_10, how='inner',left_on = left_cols,right_on=right_cols)
merge_18_10 = merge_18_10.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2018-11

In [ ]:
common_test= common_route[common_route['date'].str.contains("2018-11",case=False)]
flights_18_11 = flights[flights['fl_year_mon'].str.contains("2018-11")]
merge_18_11 = pd.merge(left=common_test, right = flights_18_11, how='inner',left_on = left_cols,right_on=right_cols)
merge_18_11 = merge_18_11.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2018-12

In [ ]:
common_test= common_route[common_route['date'].str.contains("2018-12",case=False)]
flights_18_12 = flights[flights['fl_year_mon'].str.contains("2018-12")]
merge_18_12 = pd.merge(left=common_test, right = flights_18_12, how='inner',left_on = left_cols,right_on=right_cols)
merge_18_12 = merge_18_12.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

### 2019

##### 2019-01

In [107]:
common_test= common_route[common_route['date'].str.contains("2019-1",case=False)]
common_test= common_test[common_test['date'].str.len()==6]
flights_19_01 = flights[flights['fl_year_mon'].str.contains("2019-01")]
merge_19_01 = pd.merge(left=common_test, right = flights_19_01, how='inner',left_on = left_cols,right_on=right_cols)
merge_19_01 = merge_19_01.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

In [108]:
merge_19_01

,percent_pload_pass,percent_capacity,accel_time,route,aircraft_group,fl_date,mkt_carrier,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,distance_y,arr_delay,crs_route,time_diff_crs_elapsed_route_time,speed
0,0.534374,0.712338,10193.0,HNL-LIH,6,2019-01-01,HA,293,13:03:00,13:45:00,42,102,-2,42,0,4.636364
1,0.534374,0.712338,10193.0,HNL-LIH,6,2019-01-01,HA,313,14:10:00,14:55:00,45,102,2,45,0,4.857143
2,0.534374,0.712338,10193.0,HNL-LIH,6,2019-01-01,HA,343,15:20:00,16:00:00,40,102,-1,40,0,5.100000
3,0.534374,0.712338,10193.0,HNL-LIH,6,2019-01-01,HA,363,16:20:00,17:02:00,42,102,3,42,0,4.434783
4,0.534374,0.712338,10193.0,HNL-LIH,6,2019-01-01,HA,393,17:14:00,17:55:00,41,102,1,41,0,4.636364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387382,0.565686,0.753891,9181.0,KOA-HNL,6,2019-01-31,HA,137,07:46:00,08:28:00,42,163,-5,42,0,5.258065
1387383,0.565686,0.753891,9181.0,KOA-HNL,6,2019-01-31,HA,177,09:47:00,10:35:00,48,163,-15,48,0,5.821429
1387384,0.565686,0.753891,9181.0,KOA-HNL,6,2019-01-31,HA,237,11:59:00,12:45:00,46,163,-14,46,0,5.433333
1387385,0.565686,0.753891,9181.0,KOA-HNL,6,2019-01-31,HA,257,12:38:00,13:25:00,47,163,-2,47,0,5.093750


##### 2019-02

In [ ]:
common_test= common_route[common_route['date'].str.contains("2019-2",case=False)]
flights_19_02 = flights[flights['fl_year_mon'].str.contains("2019-02")]
merge_19_02 = pd.merge(left=common_test, right = flights_19_02, how='inner',left_on = left_cols,right_on=right_cols)
merge_19_02 = merge_19_02.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2019-03

In [ ]:
common_test= common_route[common_route['date'].str.contains("2019-3",case=False)]
flights_19_03 = flights[flights['fl_year_mon'].str.contains("2019-03")]
merge_19_03 = pd.merge(left=common_test, right = flights_19_03, how='inner',left_on = left_cols,right_on=right_cols)
merge_19_03 = merge_19_03.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2019-04

In [ ]:
common_test= common_route[common_route['date'].str.contains("2019-4",case=False)]
flights_19_04 = flights[flights['fl_year_mon'].str.contains("2019-04")]
merge_19_04 = pd.merge(left=common_test, right = flights_19_04, how='inner',left_on = left_cols,right_on=right_cols)
merge_19_04 = merge_19_04.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

In [ ]:
merge_19_04

##### 2019-05

In [ ]:
common_test= common_route[common_route['date'].str.contains("2019-5",case=False)]
flights_19_05 = flights[flights['fl_year_mon'].str.contains("2019-05")]
merge_19_05 = pd.merge(left=common_test, right = flights_19_05, how='inner',left_on = left_cols,right_on=right_cols)
merge_19_05 = merge_19_05.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2019-06

In [ ]:
common_test= common_route[common_route['date'].str.contains("2019-6",case=False)]
flights_19_06 = flights[flights['fl_year_mon'].str.contains("2019-06")]
merge_19_06 = pd.merge(left=common_test, right = flights_19_06, how='inner',left_on = left_cols,right_on=right_cols)
merge_19_06 = merge_19_06.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2019-07

In [ ]:
common_test= common_route[common_route['date'].str.contains("2019-7",case=False)]
flights_19_07 = flights[flights['fl_year_mon'].str.contains("2019-07")]
merge_19_07= pd.merge(left=common_test, right = flights_19_07, how='inner',left_on = left_cols,right_on=right_cols)
merge_19_07 = merge_19_07.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2019-08

In [ ]:
common_test= common_route[common_route['date'].str.contains("2019-8",case=False)]
flights_19_08 = flights[flights['fl_year_mon'].str.contains("2019-08")]
merge_19_08= pd.merge(left=common_test, right = flights_19_08, how='inner',left_on = left_cols,right_on=right_cols)
merge_19_08 = merge_19_08.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2019-09

In [ ]:
common_test= common_route[common_route['date'].str.contains("2019-9",case=False)]
flights_19_09 = flights[flights['fl_year_mon'].str.contains("2019-09")]
merge_19_09 = pd.merge(left=common_test, right = flights_19_09, how='inner',left_on = left_cols,right_on=right_cols)
merge_19_09 = merge_19_09.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2019-10

In [ ]:
common_test= common_route[common_route['date'].str.contains("2019-10",case=False)]
flights_19_10 = flights[flights['fl_year_mon'].str.contains("2019-10")]
merge_19_10 = pd.merge(left=common_test, right = flights_19_10, how='inner',left_on = left_cols,right_on=right_cols)
merge_19_10 = merge_19_10.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2019-11

In [ ]:
common_test= common_route[common_route['date'].str.contains("2019-11",case=False)]
flights_19_11 = flights[flights['fl_year_mon'].str.contains("2019-11")]
merge_19_11 = pd.merge(left=common_test, right = flights_19_11, how='inner',left_on = left_cols,right_on=right_cols)
merge_19_11 = merge_19_11.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

##### 2019-12

In [ ]:
common_test= common_route[common_route['date'].str.contains("2019-12",case=False)]
flights_19_12 = flights[flights['fl_year_mon'].str.contains("2019-12")]
merge_19_12 = pd.merge(left=common_test, right = flights_19_12, how='inner',left_on = left_cols,right_on=right_cols)
merge_19_12 = merge_19_12.drop(columns = ['unique_carrier','origin_airport_id','dest_airport_id','date','fl_year_mon','distance_x'])

## Delete original flights,passengers, and filtered dataframe to free up memory

In [78]:
del flights, passengers, common_carrier,common_or_id,common_des_id,common_route

## Writing to csv

In [80]:
# write df1 content in file.csv
merge_18_01.to_csv('merged.csv', index=False)
# append df2 content to file.csv
merge_18_02.to_csv('merged.csv', mode='a', columns=None, index=False)
# append df3 content to file.csv
merge_18_03.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_18_04.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_18_05.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_18_06.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_18_07.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_18_08.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_18_09.to_csv('merged.csv', mode='a', columns=None, index=False)\

merge_18_10.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_18_11.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_18_12.to_csv('merged.csv', mode='a', columns=None, index=False)

# free memory
del merge_18_01, merge_18_02, merge_18_03,merge_18_04,merge_18_05,merge_18_06,merge_18_07,merge_18_08,merge_18_09,merge_18_10,merge_18_11,merge_18_12

In [81]:
merge_19_01.to_csv('merged.csv',  mode='a',columns=None, index=False)

merge_19_02.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_19_03.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_19_04.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_19_05.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_19_06.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_19_07.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_19_08.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_19_09.to_csv('merged.csv', mode='a', columns=None, index=False)\

merge_19_10.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_19_11.to_csv('merged.csv', mode='a', columns=None, index=False)

merge_19_12.to_csv('merged.csv', mode='a', columns=None, index=False)

# free memory
del merge_19_01, merge_19_02, merge_19_03,merge_19_04,merge_19_05,merge_19_06,merge_19_07,merge_19_08,merge_19_09,merge_19_10,merge_19_11,merge_19_12

In [82]:
# read all df1, df2, df3 contents
df_all = pd.read_csv('merged.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (0,1,2,4,7,10,11,12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [83]:
print('merged data size: {}'.format(df_all.shape))

merged data size: (33127400, 16)


In [84]:
df_all.head(2)

,percent_pload_pass,percent_capacity,accel_time,route,aircraft_group,fl_date,mkt_carrier,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,distance_y,arr_delay,crs_route,time_diff_crs_elapsed_route_time,speed
0,0.676118,0.805369,27,DSM-MSP,6,2018-01-01,DL,3405,13:20:00,14:42:00,82,232,92,82,0,5.52381
1,0.676118,0.805369,27,DSM-MSP,6,2018-01-01,DL,3756,06:45:00,08:09:00,84,232,244,84,0,5.39535


In [85]:
df_all.columns

Index(['percent_pload_pass', 'percent_capacity', 'accel_time', 'route',
       'aircraft_group', 'fl_date', 'mkt_carrier', 'mkt_carrier_fl_num',
       'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time', 'distance_y',
       'arr_delay', 'crs_route', 'time_diff_crs_elapsed_route_time', 'speed'],
      dtype='object')